# Test - Comparison LLM vs BERT+LLM
Code to compare and evaluate wich is the better solution among:
1. **LLM DeepSeekV3**
2. **Finetuned BioClinicalBERT + LLM**

## Imports and environment setup

In [ ]:
!pip install pdfplumber -q
!pip install together -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.6/91.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 6.2 MB/s eta 0:00:00


In [ ]:
import pdfplumber
import os
import numpy as np
import pandas as pd

import torch
from transformers import AutoTokenizer, AutoModel,AutoModelForTokenClassification
import together
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import logging
logging.getLogger('pdfminer').setLevel(logging.ERROR)

In [ ]:
from together import Together
client = Together()

## Constants

In [ ]:
DATA_PATH='/content/drive/MyDrive/SanRaffaele/Data/Dataset TEST pdf/Lettera di dimissione.pdf'

In [ ]:
MODEL_PATH='/content/drive/MyDrive/SanRaffaele/Model/bioclinicalbert-ner-final'

In [ ]:
prompt_classificazione='''
Sei un medico specializzato in cardiochirurgia. Il tuo compito è classificare le seguenti entità TARGET nell'input ad una delle seguenti entità.
l'input sarà l'output di un modello BERT addestrato per NER, per questo motivo le label nell'input avranno formato BIO.

###**Entità solo queste):**

### Mappa delle entità e tipi

| Entità                                      | Tipo              | Descrizione                                                                                                                                     |
|--------------------------------------------|-------------------|-------------------------------------------------------------------------------------------------------------------------------------------------|
| n_cartella                                 | Number            | Numero identificativo univoco assegnato alla cartella clinica del paziente.                                                                    |
| data_ingresso_cch                          | Date              | Data in cui il paziente è stato ricoverato presso il reparto di Cardiochirurgia.                                                              |
| data_dimissione_cch                        | Date              | Data in cui il paziente è stato dimesso dal reparto di Cardiochirurgia.                                                                        |
| nome                                       | Text              | Nome proprio del paziente.                                                                                                                      |
| cognome                                    | Text              | Cognome del paziente.                                                                                                                           |
| sesso                                      | Categorical_MF    | Sesso biologico del paziente (M = Maschio, F = Femmina).                                                                                        |
| numero di telefono                         | Text              | Recapito telefonico del paziente o di un contatto di riferimento.                                                                              |
| età al momento dell'intervento             | Number            | Età del paziente calcolata alla data dell’intervento chirurgico.                                                                               |
| data_di_nascita                            | Date              | Data di nascita del paziente.                                                                                                                   |
| Diagnosi                                   | Text              | Diagnosi principale alla base dell'indicazione chirurgica.                                                                                      |
| Anamnesi                                   | Text              | Anamnesi patologica remota e prossima, utile per la valutazione del rischio operatorio.                                                        |
| Motivo ricovero                            | Text              | Indicazione clinica per il ricovero in Cardiochirurgia.                                                                                         |
| classe_nyha                                | Categorical_1234  | Classe funzionale NYHA per scompenso cardiaco (I-IV), definisce la gravità dei sintomi.                                                        |
| angor                                      | Boolean           | Presenza di angina pectoris (dolore toracico di origine ischemica).                                                                            |
| STEMI/NSTEMI                               | Boolean           | Presenza di infarto miocardico acuto con/senza sopraslivellamento del tratto ST.                                                               |
| scompenso_cardiaco_nei_3_mesi_precedenti   | Boolean           | Episodi di scompenso cardiaco documentati nei 3 mesi precedenti l’intervento.                                                                  |
| fumo                                       | Categorical_012   | Abitudine al fumo (0 = mai fumato, 1 = ex-fumatore, 2 = fumatore attivo).                                                                      |
| diabete                                    | Boolean           | Presenza di diabete mellito noto.                                                                                                               |
| ipertensione                               | Boolean           | Presenza di ipertensione arteriosa.                                                                                                             |
| dislipidemia                               | Boolean           | Presenza di dislipidemia (colesterolo e/o trigliceridi elevati).                                                                               |
| BPCO                                       | Boolean           | Presenza di broncopneumopatia cronica ostruttiva.                                                                                               |
| stroke_pregresso                           | Boolean           | Precedente episodio di ictus cerebrale ischemico o emorragico.                                                                                  |
| TIA_pregresso                              | Boolean           | Episodio pregresso di attacco ischemico transitorio (TIA).                                                                                      |
| vasculopatiaperif                          | Boolean           | Malattia vascolare periferica documentata (es. arteriopatia arti inferiori).                                                                   |
| neoplasia_pregressa                        | Boolean           | Presenza di neoplasie trattate in passato.                                                                                                      |
| irradiazionetoracica                       | Boolean           | Pregressa radioterapia al torace, rilevante per effetti tardivi su cuore e vasi.                                                               |
| insufficienza_renale_cronica               | Boolean           | Presenza di insufficienza renale cronica diagnosticata.                                                                                         |
| familiarita_cardiovascolare                | Boolean           | Familiarità per malattie cardiovascolari premature (prima dei 55 anni per uomini, 65 per donne).                                                |
| limitazione_mobilita                       | Boolean           | Presenza di limitazioni significative alla mobilità (es. pazienti allettati).                                                                  |
| endocardite                                | Boolean           | Pregressa o attiva endocardite infettiva, rilevante per indicazione chirurgica.                                                                |
| ritmo_all_ingresso                         | Categorical_012   | Ritmo cardiaco al momento del ricovero (0 = ritmo sinusale, 1 = FA, 2 = altro).                                                                 |
| fibrillazione_atriale                      | Categorical_012   | Presenza di fibrillazione atriale (0 = mai, 1 = parossistica, 2 = permanente/persistente).                                                     |
| dialisi                                    | Boolean           | Paziente in trattamento emodialitico o peritoneale.                                                                                             |
| elettivo_urgenza_emergenza                 | Categorical_012   | Tipo di intervento (0 = elettivo, 1 = urgente, 2 = emergenza).                                                                                   |
| pm                                         | Boolean           | Presenza di pacemaker.                                                                                                                          |
| crt                                        | Boolean           | Presenza di terapia di resincronizzazione cardiaca (CRT).                                                                                       |
| icd                                        | Boolean           | Presenza di defibrillatore impiantabile (ICD).                                                                                                  |
| pci_pregressa                              | Boolean           | Precedente angioplastica coronarica percutanea (PCI).                                                                                           |
| REDO                                       | Boolean           | Intervento cardiochirurgico di revisione (non prima chirurgia).                                                                                |
| Anno REDO                                  | Date              | Anno in cui è stato eseguito l'intervento REDO precedente.                                                                                      |
| Tipo di REDO                               | Text              | Descrizione del tipo di intervento REDO eseguito.                                                                                               |
| Terapia                                    | Text              | Terapia farmacologica in atto al momento del ricovero.                                                                                          |
| lasix                                      | Boolean           | Uso documentato di furosemide (Lasix).                                                                                                          |
| lasix_dosaggio                             | Number            | Dosaggio giornaliero di furosemide in mg.                                                                                                       |
| nitrati                                    | Boolean           | Assunzione di nitrati (vasodilatatori usati per l'angina).                                                                                      |
| antiaggregante                             | Boolean           | Presenza di terapia antiaggregante (es. ASA, clopidogrel).                                                                                      |
| dapt                                       | Boolean           | Doppia antiaggregazione piastrinica (es. ASA + clopidogrel/prasugrel).                                                                          |
| anticoagorali                              | Boolean           | Terapia anticoagulante in corso (es. warfarin, DOAC).                                                                                           |
| aceinib                                    | Boolean           | Uso di ACE-inibitori.                                                                                                                           |
| betabloc                                   | Boolean           | Uso di beta-bloccanti.                                                                                                                          |
| sartanici                                  | Boolean           | Uso di sartani (ARBs).                                                                                                                          |
| caantag                                    | Boolean           | Uso di calcio-antagonisti.                                                                                                                      |
| esami_all_ingresso                         | Text              | Risultati di laboratorio e strumentali al momento dell’ingresso.                                                                               |
| Decorso_post_operatorio                    | Text              | Descrizione del decorso clinico successivo all’intervento chirurgico.                                                                          |
| IABP/ECMO/IMPELLA                          | Boolean           | Necessità di supporto meccanico circolatorio (IABP, ECMO o Impella).                                                                           |
| Inotropi                                   | Boolean           | Necessità di farmaci inotropi positivi nel post-operatorio.                                                                                     |
| secondo_intervento                         | Boolean           | Esecuzione di un secondo intervento durante la degenza attuale.                                                                                |
| Tipo_secondo_intervento                    | Text              | Tipo e motivazione del secondo intervento chirurgico.                                                                                           |
| II_Run                                     | Boolean           | Presenza di secondo passaggio in circolazione extracorporea (CEC).                                                                             |
| Causa_II_Run_CEC                           | Text              | Motivazione per il secondo utilizzo della CEC.                                                                                                  |
| LCOS                                       | Boolean           | Sindrome da bassa portata cardiaca (Low Cardiac Output Syndrome) post-operatoria.                                                               |
| Impianto_PM_post_intervento                | Boolean           | Necessità di impianto di pacemaker dopo l’intervento.                                                                                           |
| Stroke_TIA_post_op                         | Boolean           | Evento neurologico ischemico (TIA/stroke) avvenuto dopo l’intervento.                                                                          |
| Necessità_di_trasfusioni                   | Boolean           | Necessità di trasfusioni ematiche post-intervento.                                                                                              |
| IRA                                        | Boolean           | Insufficienza renale acuta insorta nel post-operatorio.                                                                                         |
| Insufficienza_respiratoria                 | Boolean           | Insorgenza di insufficienza respiratoria nel post-operatorio.                                                                                   |
| FA_di_nuova_insorgenza                     | Boolean           | Fibrillazione atriale di nuova insorgenza nel post-operatorio.                                                                                  |
| Ritmo_alla_dimissione                      | Categorical_012   | Ritmo cardiaco documentato alla dimissione (0 = sinusale, 1 = FA, 2 = altro).                                                                   |
| H_Stay_giorni (da intervento a dimissione) | Number            | Durata della degenza in giorni, calcolata dall’intervento alla dimissione.                                                                      |
| Morte                                      | Boolean           | Evento di decesso durante la degenza cardiochirurgica.                                                                                          |
| Causa_morte                                | Text              | Causa clinica del decesso (es. sepsi, shock cardiogeno, ecc.).                                                                                  |
| data_morte                                 | Date              | Data del decesso, se avvenuto.                                                                                                                  |
| esami_alla_dimissione                      | Text              | Risultati di laboratorio e strumentali prima della dimissione.                                                                                  |
| terapia_alla_dimissione                    | Text              | Terapia farmacologica prescritta alla dimissione.                                                                                               |



---

### **Istruzioni IMPORTANTI:**

-Considera specialmente le parole TARGET nell'input
- Non classificare in **nessuna entità** diversa da quelle elencate.
- Se un'entità non è presente nell'input da classificare, **non inventarla** e **non includerla** nel risultato.
- Attenzione però i nomi delle entità che vedi sopra sono in alcuni casi degli acronimi o diminutivi delle entità.
- Il formato di output deve essere una lista JSON, dove ogni elemento è un oggetto con **due chiavi**:
    - `"entità"`: il nome dell'entità
    - `"valore"`: il valore estratto dell'entità
**NON** aggiungere commenti, spiegazioni, note, intestazioni o altro: **solo** la lista JSON.
---

###Esmpio output(esempio parziale in JSON):

```json
[
  { "entità": "data_dimissione_cch", "valore": "02/09/2019" },
  { "entità": "nome", "valore": "FRANCO" },
  { "entità": "cognome", "valore": "BERTOLOTTI" },
  { "entità": "data_di_nascita", "valore": "27/03/1939" },
  { "entità": "numero di telefono", "valore": "3479927663" },
  { "entità": "data_ingresso_cch", "valore": "27/08/2019" },
  { "entità": "n_cartella", "valore": "2019034139" },
  { "entità": "Diagnosi text", "valore": "Intervento di plastica valvolare mitralica per via percutanea mediante posizionamento di duplice dispositivo Mitraclip." },
  { "entità": "Motivo ricovero", "valore": "Insufficienza mitralica in status post rivascolarizzazione miocardica chirurgica mediante triplice bypass coronarico." },
  { "entità": "fumo", "valore": true },
  { "entità": "diabete", "valore": true },
  { "entità": "insufficienza renale cronica", "valore": true },
  { "entità": "familiarita cardiovascolare", "valore": true }
]

input da classificare:


'''

In [ ]:
prompt_estrazione='''
Sei un medico specializzato in cardiochirurgia. Il tuo compito è estrarre **esclusivamente** le seguenti entità dalla **lettera di dimissione** riportata qui sotto.

###**Entità da estrarre (solo queste):**

### Mappa delle entità e tipi

| Entità                                      | Tipo              | Descrizione                                                                                                                                     |
|--------------------------------------------|-------------------|-------------------------------------------------------------------------------------------------------------------------------------------------|
| n_cartella                                 | Number            | Numero identificativo univoco assegnato alla cartella clinica del paziente.                                                                    |
| data_ingresso_cch                          | Date              | Data in cui il paziente è stato ricoverato presso il reparto di Cardiochirurgia.                                                              |
| data_dimissione_cch                        | Date              | Data in cui il paziente è stato dimesso dal reparto di Cardiochirurgia.                                                                        |
| nome                                       | Text              | Nome proprio del paziente.                                                                                                                      |
| cognome                                    | Text              | Cognome del paziente.                                                                                                                           |
| sesso                                      | Categorical_MF    | Sesso biologico del paziente (M = Maschio, F = Femmina).                                                                                        |
| numero di telefono                         | Text              | Recapito telefonico del paziente o di un contatto di riferimento.                                                                              |
| età al momento dell'intervento             | Number            | Età del paziente calcolata alla data dell’intervento chirurgico.                                                                               |
| data_di_nascita                            | Date              | Data di nascita del paziente.                                                                                                                   |
| Diagnosi                                   | Text              | Diagnosi principale alla base dell'indicazione chirurgica.                                                                                      |
| Anamnesi                                   | Text              | Anamnesi patologica remota e prossima, utile per la valutazione del rischio operatorio.                                                        |
| Motivo ricovero                            | Text              | Indicazione clinica per il ricovero in Cardiochirurgia.                                                                                         |
| classe_nyha                                | Categorical_1234  | Classe funzionale NYHA per scompenso cardiaco (I-IV), definisce la gravità dei sintomi.                                                        |
| angor                                      | Boolean           | Presenza di angina pectoris (dolore toracico di origine ischemica).                                                                            |
| STEMI/NSTEMI                               | Boolean           | Presenza di infarto miocardico acuto con/senza sopraslivellamento del tratto ST.                                                               |
| scompenso_cardiaco_nei_3_mesi_precedenti   | Boolean           | Episodi di scompenso cardiaco documentati nei 3 mesi precedenti l’intervento.                                                                  |
| fumo                                       | Categorical_012   | Abitudine al fumo (0 = mai fumato, 1 = ex-fumatore, 2 = fumatore attivo).                                                                      |
| diabete                                    | Boolean           | Presenza di diabete mellito noto.                                                                                                               |
| ipertensione                               | Boolean           | Presenza di ipertensione arteriosa.                                                                                                             |
| dislipidemia                               | Boolean           | Presenza di dislipidemia (colesterolo e/o trigliceridi elevati).                                                                               |
| BPCO                                       | Boolean           | Presenza di broncopneumopatia cronica ostruttiva.                                                                                               |
| stroke_pregresso                           | Boolean           | Precedente episodio di ictus cerebrale ischemico o emorragico.                                                                                  |
| TIA_pregresso                              | Boolean           | Episodio pregresso di attacco ischemico transitorio (TIA).                                                                                      |
| vasculopatiaperif                          | Boolean           | Malattia vascolare periferica documentata (es. arteriopatia arti inferiori).                                                                   |
| neoplasia_pregressa                        | Boolean           | Presenza di neoplasie trattate in passato.                                                                                                      |
| irradiazionetoracica                       | Boolean           | Pregressa radioterapia al torace, rilevante per effetti tardivi su cuore e vasi.                                                               |
| insufficienza_renale_cronica               | Boolean           | Presenza di insufficienza renale cronica diagnosticata.                                                                                         |
| familiarita_cardiovascolare                | Boolean           | Familiarità per malattie cardiovascolari premature (prima dei 55 anni per uomini, 65 per donne).                                                |
| limitazione_mobilita                       | Boolean           | Presenza di limitazioni significative alla mobilità (es. pazienti allettati).                                                                  |
| endocardite                                | Boolean           | Pregressa o attiva endocardite infettiva, rilevante per indicazione chirurgica.                                                                |
| ritmo_all_ingresso                         | Categorical_012   | Ritmo cardiaco al momento del ricovero (0 = ritmo sinusale, 1 = FA, 2 = altro).                                                                 |
| fibrillazione_atriale                      | Categorical_012   | Presenza di fibrillazione atriale (0 = mai, 1 = parossistica, 2 = permanente/persistente).                                                     |
| dialisi                                    | Boolean           | Paziente in trattamento emodialitico o peritoneale.                                                                                             |
| elettivo_urgenza_emergenza                 | Categorical_012   | Tipo di intervento (0 = elettivo, 1 = urgente, 2 = emergenza).                                                                                   |
| pm                                         | Boolean           | Presenza di pacemaker.                                                                                                                          |
| crt                                        | Boolean           | Presenza di terapia di resincronizzazione cardiaca (CRT).                                                                                       |
| icd                                        | Boolean           | Presenza di defibrillatore impiantabile (ICD).                                                                                                  |
| pci_pregressa                              | Boolean           | Precedente angioplastica coronarica percutanea (PCI).                                                                                           |
| REDO                                       | Boolean           | Intervento cardiochirurgico di revisione (non prima chirurgia).                                                                                |
| Anno REDO                                  | Date              | Anno in cui è stato eseguito l'intervento REDO precedente.                                                                                      |
| Tipo di REDO                               | Text              | Descrizione del tipo di intervento REDO eseguito.                                                                                               |
| Terapia                                    | Text              | Terapia farmacologica in atto al momento del ricovero.                                                                                          |
| lasix                                      | Boolean           | Uso documentato di furosemide (Lasix).                                                                                                          |
| lasix_dosaggio                             | Number            | Dosaggio giornaliero di furosemide in mg.                                                                                                       |
| nitrati                                    | Boolean           | Assunzione di nitrati (vasodilatatori usati per l'angina).                                                                                      |
| antiaggregante                             | Boolean           | Presenza di terapia antiaggregante (es. ASA, clopidogrel).                                                                                      |
| dapt                                       | Boolean           | Doppia antiaggregazione piastrinica (es. ASA + clopidogrel/prasugrel).                                                                          |
| anticoagorali                              | Boolean           | Terapia anticoagulante in corso (es. warfarin, DOAC).                                                                                           |
| aceinib                                    | Boolean           | Uso di ACE-inibitori.                                                                                                                           |
| betabloc                                   | Boolean           | Uso di beta-bloccanti.                                                                                                                          |
| sartanici                                  | Boolean           | Uso di sartani (ARBs).                                                                                                                          |
| caantag                                    | Boolean           | Uso di calcio-antagonisti.                                                                                                                      |
| esami_all_ingresso                         | Text              | Risultati di laboratorio e strumentali al momento dell’ingresso.                                                                               |
| Decorso_post_operatorio                    | Text              | Descrizione del decorso clinico successivo all’intervento chirurgico.                                                                          |
| IABP/ECMO/IMPELLA                          | Boolean           | Necessità di supporto meccanico circolatorio (IABP, ECMO o Impella).                                                                           |
| Inotropi                                   | Boolean           | Necessità di farmaci inotropi positivi nel post-operatorio.                                                                                     |
| secondo_intervento                         | Boolean           | Esecuzione di un secondo intervento durante la degenza attuale.                                                                                |
| Tipo_secondo_intervento                    | Text              | Tipo e motivazione del secondo intervento chirurgico.                                                                                           |
| II_Run                                     | Boolean           | Presenza di secondo passaggio in circolazione extracorporea (CEC).                                                                             |
| Causa_II_Run_CEC                           | Text              | Motivazione per il secondo utilizzo della CEC.                                                                                                  |
| LCOS                                       | Boolean           | Sindrome da bassa portata cardiaca (Low Cardiac Output Syndrome) post-operatoria.                                                               |
| Impianto_PM_post_intervento                | Boolean           | Necessità di impianto di pacemaker dopo l’intervento.                                                                                           |
| Stroke_TIA_post_op                         | Boolean           | Evento neurologico ischemico (TIA/stroke) avvenuto dopo l’intervento.                                                                          |
| Necessità_di_trasfusioni                   | Boolean           | Necessità di trasfusioni ematiche post-intervento.                                                                                              |
| IRA                                        | Boolean           | Insufficienza renale acuta insorta nel post-operatorio.                                                                                         |
| Insufficienza_respiratoria                 | Boolean           | Insorgenza di insufficienza respiratoria nel post-operatorio.                                                                                   |
| FA_di_nuova_insorgenza                     | Boolean           | Fibrillazione atriale di nuova insorgenza nel post-operatorio.                                                                                  |
| Ritmo_alla_dimissione                      | Categorical_012   | Ritmo cardiaco documentato alla dimissione (0 = sinusale, 1 = FA, 2 = altro).                                                                   |
| H_Stay_giorni (da intervento a dimissione) | Number            | Durata della degenza in giorni, calcolata dall’intervento alla dimissione.                                                                      |
| Morte                                      | Boolean           | Evento di decesso durante la degenza cardiochirurgica.                                                                                          |
| Causa_morte                                | Text              | Causa clinica del decesso (es. sepsi, shock cardiogeno, ecc.).                                                                                  |
| data_morte                                 | Date              | Data del decesso, se avvenuto.                                                                                                                  |
| esami_alla_dimissione                      | Text              | Risultati di laboratorio e strumentali prima della dimissione.                                                                                  |
| terapia_alla_dimissione                    | Text              | Terapia farmacologica prescritta alla dimissione.                                                                                               |



---

### **Istruzioni IMPORTANTI:**

- Ragiona considerando **frase per frase**.
- Non estrarre **nessuna entità** diversa da quelle elencate.
- Se un'entità non è presente nella lettera, **non inventarla** e **non includerla** nel risultato.
- Attenzione però i nomi delle entità che vedi sopra sono in alcuni casi degli acronimi o diminutivi delle entità.
- Il formato di output deve essere una lista JSON, dove ogni elemento è un oggetto con **due chiavi**:
    - `"entità"`: il nome dell'entità
    - `"valore"`: il valore estratto dell'entità
**NON** aggiungere commenti, spiegazioni, note, intestazioni o altro: **solo** la lista JSON.

---

###Esempio di input(esempio parziale della lettera di dimission)
Si dimette in data 02/09/2019
il Sig. BERTOLOTTI FRANCO
Nato il 27/03/1939 telefono 3479927663
ricoverato presso questo ospedale dal 27/08/2019
Numero Cartella 2019034139

Diagnosi alla dimissione:
Intervento di plastica valvolare mitralica per via percutanea mediante posizionamento di duplice dispositivo Mitraclip.

Motivo del Ricovero:
Insufficienza mitralica in status post rivascolarizzazione miocardica chirurgica mediante triplice bypass coronarico.

Cenni Anamnestici:
Paziente nega farmacoallergie.
Familiarità positiva per cardiopatia ischemica (padre).
Ex fumatore, stop nel 1990 (1 pack/die).
Diabete mellito in tp ipoglicemizzante orale.
IRC (crea all'ingresso 2,64 mg/dl).


---

###Esmpio output(esempio parziale in JSON):

```json
[
  { "entità": "data_dimissione_cch", "valore": "02/09/2019" },
  { "entità": "nome", "valore": "FRANCO" },
  { "entità": "cognome", "valore": "BERTOLOTTI" },
  { "entità": "data_di_nascita", "valore": "27/03/1939" },
  { "entità": "numero di telefono", "valore": "3479927663" },
  { "entità": "data_ingresso_cch", "valore": "27/08/2019" },
  { "entità": "n_cartella", "valore": "2019034139" },
  { "entità": "Diagnosi text", "valore": "Intervento di plastica valvolare mitralica per via percutanea mediante posizionamento di duplice dispositivo Mitraclip." },
  { "entità": "Motivo ricovero", "valore": "Insufficienza mitralica in status post rivascolarizzazione miocardica chirurgica mediante triplice bypass coronarico." },
  { "entità": "fumo", "valore": true },
  { "entità": "diabete", "valore": true },
  { "entità": "insufficienza renale cronica", "valore": true },
  { "entità": "familiarita cardiovascolare", "valore": true }
]

lettera di dimissione:


'''

## Functions

In [ ]:
def extract_text(file,num_pag):
  with pdfplumber.open(DATA_PATH) as pdf:
      page=pdf.pages[num_pag]
      all_text=page.extract_text()
      return all_text

In [ ]:
def extract_tables(file,num_pag):
  with pdfplumber.open(DATA_PATH) as pdf:
      page=pdf.pages[num_pag]
      all_tables=page.extract_tables()
      return all_tables

In [ ]:
def get_text_to_remove(all_tables):
  text_to_remove=[]
  if (len(all_tables)>1):
    for i in range(0,len(all_tables)):
      all_tables[i]=np.array(all_tables[i])
      text_to_remove.extend([str(" ".join(row)).replace(" ", "").replace('\n', "") for row in all_tables[i]])
  else:
    all_tables=np.array(all_tables)
    all_tables_=all_tables.reshape(-1,np.shape(all_tables)[-1])
    text_to_remove.extend([str(" ".join(row)).replace(" ", "").replace('\n', "") for row in all_tables_])
  return text_to_remove

In [ ]:
def remove_tables(all_text,text_to_remove):
  clean_text=all_text
  for row in clean_text.split('\n'):
    row_=row.replace(" ", "")
    row_=row_.replace('\n', "")
    if row_ in text_to_remove:
          clean_text = clean_text.replace(row+'\n', '')
  return clean_text

#GESTIRE CASO LIM EMOCROMO - MCHC (Conc.Emoglobinica 341 g/L 310 - 360 20/01/2025 Corpusc.Media)
#Non viene eliminato perchè extract text and extract tables lo trascivono i 2 modi diversi, ectract tables è qiello giusto
#Problema quando nella cella dell'excel va a capo

In [ ]:
def get_cleaned_text(all_text,all_tables):
  if len(all_tables)>0:
    text_to_remove=get_text_to_remove(all_tables)
    cleaned_text=remove_tables(all_text,text_to_remove)
  else:
    cleaned_text=all_text
  return cleaned_text

In [ ]:
def extract_all_pages(DATA_PATH):
  list_cleaned_text=[]
  with pdfplumber.open(DATA_PATH) as pdf:
    num_pages= len(pdf.pages) #number of pages in
    print(num_pages)

    for i in range(num_pages):
      print("Page: "+str(i))
      all_text=extract_text(DATA_PATH,i)
      all_tables=extract_tables(DATA_PATH,i)
      print(f"Tables:{all_tables}")
      cleaned_text=get_cleaned_text(all_text,all_tables)
      list_cleaned_text.append(cleaned_text)
  return list_cleaned_text

In [ ]:
def split_into_sentences(text: str) -> list:
    # Separazione in frasi
    sentences = text.split('\n')

    # Pulizia finale di ogni frase
    sentences = [s.strip() for s in sentences if s.strip()]

    return sentences

In [ ]:
def get_response(prompt_text:str, model:str)->str:
  try:
    response =  client.chat.completions.create(
          model=model,
          messages=[{"role": "user", "content": prompt_text}],
          response_format={"type": "json_schema"},
          temperature=0.1,
          top_p=1,
          max_tokens=8192
          )

  except together.error.RateLimitError as e:
      print(e)

  response_text=response.choices[0].message.content
  return response_text

In [ ]:
def create_dataframe(response_text:str, )->pd.DataFrame:
  json_response=json.loads(response_text)
  df = pd.DataFrame(json_response)
  return df

In [ ]:
def predict_ner_tags(sentence):
    words = sentence.split()

    # 1. Tokenizzazione senza tensori per usare .word_ids()
    tokenized_inputs = tokenizer(
        words,
        is_split_into_words=True,
        truncation=True,
        padding="max_length",
        max_length=128,
        return_tensors=None
    )

    word_ids = tokenized_inputs.word_ids()

    # 2. Converti in tensori per PyTorch
    torch_inputs = tokenizer(
        words,
        is_split_into_words=True,
        truncation=True,
        padding="max_length",
        max_length=128,
        return_tensors="pt"
    )

    with torch.no_grad():
        outputs = model(**torch_inputs)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1).squeeze().tolist()


    final_predictions = []
    previous_word_id = None

    for idx, word_id in enumerate(word_ids):
        if word_id is None or word_id == previous_word_id:
            continue
        label_id = predictions[idx]
        label = id_to_label[label_id]

        final_predictions.append((words[word_id], label))
        previous_word_id = word_id

    return final_predictions


## Extraction text

In [ ]:
list_cleaned_text_all=extract_all_pages(DATA_PATH)

7
Page: 0
Tables:[]
Page: 1
Tables:[]
Page: 2
Tables:[[['Gruppo sanguigno', '0', '', '', '20/01/2025'], ['Fattore RH', 'Positivo', '', '', '20/01/2025'], ['Ricerca Anticorpi Irregolari', 'Negativo', '', '', '20/01/2025'], ['Numero riferimento', '2502542', '', '', '20/01/2025'], ['EMOCROMO - Globuli bianchi', '6.5', '10^9/L', '4.8 - 10.8', '20/01/2025'], ['EMOCROMO - Globuli rossi', '4.78', '10^12/L', '4.7 - 6.1', '20/01/2025'], ['EMOCROMO - Ematocrito', '43.4', '%', '42 - 52', '20/01/2025'], ['EMOCROMO - MCV (Volume Globulare Medio)', '90.8', 'fL', '80 - 94', '20/01/2025'], ['EMOCROMO - MCH (Emoglobina Corpuscolare Media)', '31.0', 'pg/Cell', '27 - 34', '20/01/2025'], ['EMOCROMO - RDW (Ind.Distribuzione Eritrocitaria)', '12.6', 'CV %', '11.5 - 14.5', '20/01/2025'], ['PIASTRINE - Piastrine', '180', '10^9/L', '130 - 400', '20/01/2025'], ['PIASTRINE - MPV (Volume Piastrinico Medio)', '9.6', 'fL', '9.1 - 12.5', '20/01/2025'], ['FORMULA - Neutrofili', '56.1', '%', '40 - 75', '20/01/2025'], 

In [ ]:
text=list_cleaned_text_all[0]

In [ ]:
sentences=split_into_sentences(text)

## Inference BIOClinicalBERT

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model= AutoModelForTokenClassification .from_pretrained(MODEL_PATH)

In [ ]:
label_to_id = {'O': 0, 'B-TARGET': 1, 'I-TARGET': 2}

id_to_label = {v: k for k, v in label_to_id.items()}

In [ ]:
import pandas as pd

df = pd.DataFrame(columns=['frase', 'label'])
list_tuple=[]
for sentence in sentences:
    ner_output = predict_ner_tags(sentence)  # Lista di tuple (word, label)
    list_tuple.append(ner_output)
    words = " ".join([word for word, label in ner_output])
    labels = [label for word, label in ner_output]

    # Aggiungi nuova riga al DataFrame
    df.loc[len(df)] = [words, labels]


In [ ]:
str(list_tuple)

'[[(\'Dipartimento\', \'O\'), (\'Cardiotoracovascolare\', \'O\')], [(\'U.O.\', \'O\'), (\'Cardiochirurgia\', \'O\')], [(\'Direttore\', \'O\'), (\'Prof.\', \'O\'), (\'Francesco\', \'B-TARGET\'), (\'Maisano\', \'I-TARGET\')], [(\'RELAZIONE\', \'O\'), (\'CLINICA\', \'O\'), (\'alla\', \'O\'), (\'DIMISSIONE\', \'O\'), (\'-\', \'O\'), (\'Definitiva\', \'O\')], [(\'Milano,\', \'O\'), (\'il\', \'O\'), (\'27/01/2025\', \'B-TARGET\')], [(\'Alla\', \'O\'), (\'cortese\', \'O\'), (\'attenzione\', \'O\'), (\'del\', \'O\'), (\'medico\', \'O\'), (\'curante\', \'O\')], [(\'Si\', \'O\'), (\'dimette\', \'O\'), (\'in\', \'O\'), (\'data\', \'O\'), (\'27/01/2025\', \'B-TARGET\')], [(\'il\', \'O\'), (\'Sig.\', \'O\'), (\'RICCA\', \'B-TARGET\'), (\'MASSIMO\', \'I-TARGET\')], [(\'Nato\', \'O\'), (\'il\', \'O\'), (\'17/02/1966\', \'B-TARGET\'), (\'telefono\', \'O\'), (\'3287351755\', \'B-TARGET\'), (\'Numero\', \'O\'), (\'Cartella\', \'O\'), (\'2025003002\', \'B-TARGET\')], [(\'ricoverato\', \'O\'), (\'presso\'

In [ ]:
df

,frase,label
0,Dipartimento Cardiotoracovascolare,"[O, O]"
1,U.O. Cardiochirurgia,"[O, O]"
2,Direttore Prof. Francesco Maisano,"[O, O, B-TARGET, I-TARGET]"
3,RELAZIONE CLINICA alla DIMISSIONE - Definitiva,"[O, O, O, O, O, O]"
4,"Milano, il 27/01/2025","[O, O, B-TARGET]"
5,Alla cortese attenzione del medico curante,"[O, O, O, O, O, O]"
6,Si dimette in data 27/01/2025,"[O, O, O, O, B-TARGET]"
7,il Sig. RICCA MASSIMO,"[O, O, B-TARGET, I-TARGET]"
8,Nato il 17/02/1966 telefono 3287351755 Numero ...,"[O, O, B-TARGET, O, B-TARGET, O, O, B-TARGET]"
9,ricoverato presso questo ospedale dal 20/01/2025,"[O, O, O, O, O, B-TARGET]"


In [ ]:
prompt=prompt_classificazione+str(list_tuple)
response_BERT_LLM=get_response(prompt, 'deepseek-ai/DeepSeek-V3')

## Test BERT + LLM

In [ ]:
df_BERT_LLM=create_dataframe(response_BERT_LLM)
df_BERT_LLM

,entità,valore
0,data_dimissione_cch,27/01/2025
1,nome,MASSIMO
2,cognome,RICCA
3,data_di_nascita,17/02/1966
4,numero di telefono,3287351755
5,n_cartella,2025003002
6,data_ingresso_cch,20/01/2025
7,Diagnosi,Sostituzione valvolare aortica con protesi bio...
8,Motivo ricovero,Stenosi aortica
9,familiarita_cardiovascolare,True


## Test LLM

In [ ]:
prompt=prompt_estrazione+'\n'+text
response_LLM=get_response(prompt, 'deepseek-ai/DeepSeek-V3')

In [ ]:
df_LLM=create_dataframe(response_LLM)
df_LLM

,entità,valore
0,data_dimissione_cch,27/01/2025
1,nome,MASSIMO
2,cognome,RICCA
3,data_di_nascita,17/02/1966
4,numero di telefono,3287351755
5,n_cartella,2025003002
6,data_ingresso_cch,20/01/2025
7,Diagnosi,In data 21/01/2025 Sostituzione valvolare aort...
8,Motivo ricovero,Stenosi aortica
9,familiarita_cardiovascolare,True
